In [1]:
import os
import re
import tempfile

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.atomate2.agents import MLFFMDAgent
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)


# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
OPENAI_GPT_MODEL = "gpt-4-turbo"

In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0.0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool(
    description=(
        "A Python shell. Use this to execute python commands. "
        "Input should be a valid python command. "
        "If you want to see the output of a value, you should print it out "
        "with `print(...)`."
        "If you write the output to a file, ALWAYS print the path to the file."
    )
)]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MLFFMDAgent(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
    r"\s+",
    " ",
    """You name is LLaMP and you are a helpful agent that can consult 
    materials-related data through Materials Project (MP) database, arXiv, 
    Wikipedia, and run calculations/simulations via a python REPL and MP atomate2 
    computational workflow.     
        
    If you get an error, debug your code and try again. Only use 
    the output of your code to answer the question. Ask user to clarify their 
    queries if needed. Please note that you don't have direct control over MP but 
    through multiple assistant agents to help you. You need to provide complete 
    context in the input for assistants to do their job. REFINE and CRITQUE the 
    output of the assistants if needed. You can end the conversation by saying "Bye".
    """,
    )
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:

pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

query = """
I would like you to help me run MD simulation with the follwing steps:

1. Retrieve the stable LiTaO3 structure from Materials Project
2. Create a supercell around 100 atoms
2. Use atomate2 with foundation ML forcefield MACE to run Nose-Hoover NVT simulation from 300K to 800K for 1 ps
3. Save the trajectory file
4. Read the trajectory file and plot the temperature vs time
"""

with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
    os.chdir(tmpdir)
    while True:
        query = query or input("Query:")
        print(f"Query: {query}")
        response = agent_executor.invoke(
            {
                "input": query,
            }
        )
        if re.search(pattern, response["output"]):
            break
        query = None

Query: 
I would like you to help me run MD simulation with the follwing steps:

1. Retrieve the stable LiTaO3 structure from Materials Project
2. Create a supercell around 100 atoms
2. Use atomate2 with foundation ML forcefield MACE to run Nose-Hoover NVT simulation from 300K to 800K for 1 ps
3. Save the trajectory file
4. Read the trajectory file and plot the temperature vs time



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPStructureRetriever
Action Input: LiTaO3Thought: Do I need to use a tool? Yes
Action: MPStructureRetriever
Action Input: LiTaO3

> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "search_materials_structure__get",
  "action_input": {
    "formula": "LiTaO3",
    "limit": 5,
    "fields": "material_id,structure"
  }
}
```Action:
```json
{
  "action": "search_materials_structure__get",
  "action_input": {
    "formula": "LiTaO3",
    "limit": 5,
    "fields": "material_id,structure"
  }
}
```

Retrieving SummaryDoc documents:   0%|          | 0/5 [00:00<?, ?it/s]

All retrieved structures are saved as Pymatgen Structure json files to the following paths: mp-3666.json, mp-1105280.json, mp-754345.json, mp-1105216.json, mp-1105326.json


> Finished chain.
{'input': 'LiTaO3', 'output': 'All retrieved structures are saved as Pymatgen Structure json files to the following paths: mp-3666.json, mp-1105280.json, mp-754345.json, mp-1105216.json, mp-1105326.json'}Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: mp-3666Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: mp-3666

> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "search_materials_summary__get",
  "action_input": {
    "material_ids": "mp-3666"
  }
}
```Action:
```json
{
  "action": "search_materials_summary__get",
  "action_input": {
    "material_ids": "mp-3666"
  }
}
```Error on search_materials_summary__get: `fields` must be specified in the query. Please revise arguments or try smaller request by specifying 'limit' in request.Action:
```

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'nsites': 10, 'composition': {'Li': 2.0, 'Ta': 2.0, 'O': 6.0}, 'formula_pretty': 'LiTaO3', 'symmetry': {'crystal_system': 'Trigonal', 'symbol': 'R3c', 'number': 161, 'point_group': '3m', 'symprec': 0.1, 'version': '2.0.2'}, 'material_id': 'mp-3666'}]Action:
```json
{
  "action": "Final Answer",
  "action_input": "The material with ID mp-3666 is Lithium Tantalate (LiTaO3). It has a trigonal crystal system with space group R3c (number 161) and point group 3m. The structure consists of 10 sites with a composition of Li2Ta2O6."
}
```Action:
```json
{
  "action": "Final Answer",
  "action_input": "The material with ID mp-3666 is Lithium Tantalate (LiTaO3). It has a trigonal crystal system with space group R3c (number 161) and point group 3m. The structure consists of 10 sites with a composition of Li2Ta2O6."
}
```

> Finished chain.
{'input': 'mp-3666', 'output': 'The material with ID mp-3666 is Lithium Tantalate (LiTaO3). It has a trigonal crystal system with space group R3c (number 161)

...

All retrieved structures are saved as Pymatgen Structure json files to the following paths: mp-3666.json


> Finished chain.
{'input': 'mp-3666', 'output': 'All retrieved structures are saved as Pymatgen Structure json files to the following paths: mp-3666.json'}Do I need to use a tool? Yes
Action: Python_REPL
Action Input: 
```python
from pymatgen.core import Structure

# Load the structure from the JSON file
structure = Structure.from_file('mp-3666.json')

# Create a supercell to approximately have around 100 atoms
num_atoms = len(structure)
scaling_factor = (100 / num_atoms) ** (1/3)
supercell = structure * scaling_factor

# Save the supercell structure to a file
supercell.to(filename='supercell.json')
print('Supercell structure saved to supercell.json')
```

Python REPL can execute arbitrary code. Use with caution.


Do I need to use a tool? Yes
Action: Python_REPL
Action Input: 
```python
from pymatgen.core import Structure

# Load the structure from the JSON file
structure = Structure.from_file('mp-3666.json')

# Create a supercell to approximately have around 100 atoms
num_atoms = len(structure)
scaling_factor = (100 / num_atoms) ** (1/3)
supercell = structure * scaling_factor

# Save the supercell structure to a file
supercell.to(filename='supercell.json')
print('Supercell structure saved to supercell.json')
```Supercell structure saved to supercell.json
Do I need to use a tool? Yes
Action: MLFFMDAgent
Action Input: {"structure_path": "supercell.json", "force_field": "MACE", "simulation_type": "NVT", "temperature_range": [300, 800], "simulation_time": 1, "ensemble": "Nose-Hoover"}Do I need to use a tool? Yes
Action: MLFFMDAgent
Action Input: {"structure_path": "supercell.json", "force_field": "MACE", "simulation_type": "NVT", "temperature_range": [300, 800], "simulation_time": 1, "ensemble": "N

Default dtype float32 does not match model dtype float64, converting models to float32.


KeyboardInterrupt: 

In [ ]:
response